<a href="https://colab.research.google.com/github/Anilesh05/Big_Data_Laboratory/blob/main/15_Find_Frequency_of_Books_Published.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk
!wget https://dlcdn.apache.org/hadoop/common/hadoop-3.3.6/hadoop-3.3.6.tar.gz
!tar fx hadoop-3.3.6.tar.gz
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["HADOOP_HOME"] = "/content/hadoop-3.3.6"
!ln -s /content/hadoop-3.3.6/bin/* /usr/bin

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java libatk-wrapper-java-jni libfontenc1
  libgail-common libgail18 libgtk2.0-0 libgtk2.0-bin libgtk2.0-common libice-dev librsvg2-common
  libsm-dev libxkbfile1 libxt-dev libxtst6 libxxf86dga1 openjdk-8-jdk-headless openjdk-8-jre
  openjdk-8-jre-headless x11-utils
Suggested packages:
  gvfs libice-doc libsm-doc libxt-doc openjdk-8-demo openjdk-8-source visualvm libnss-mdns
  fonts-nanum fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei
  fonts-indic mesa-utils
The following NEW packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java libatk-wrapper-java-jni libfontenc1
  libgail-common libgail18 libgtk2.0-0 libgtk2.0-bin libgtk2.0-common libice-dev librsvg2-common
  libsm-dev libxkbfile1 libxt-dev libxtst6 libxxf86dga1 openjdk-

In [2]:
%%writefile mapper.py
#!/usr/bin/env python
import sys
# Input comes from STDIN (standard input)
for line in sys.stdin:
    # Remove leading and trailing whitespace
    line = line.strip()
    # Split the line into words
    words = line.split(",")
    # Emit year and 1 to the output
    print(f"{words[3]}\t1")

Writing mapper.py


In [3]:
%%writefile reducer.py
#!/usr/bin/env python
import sys

current_year, current_count, max_year, max_count = None, 0, None, 0

print("Year\tCount")  # Add column names
print()

for line in sys.stdin:
    year, count = line.strip().split('\t')
    try:
        count = int(count)
    except ValueError:
        continue
    if current_year == year:
        current_count += count
    else:
        if current_year:
            print(f"{current_year}\t{current_count}")
            if current_count > max_count:
                max_count, max_year = current_count, current_year
        current_year, current_count = year, count

if current_year:
    print(f"{current_year}\t{current_count}")

if max_year:
    print()
    print(f"Year with maximum books published: {max_year} (no.of.books : {max_count})")


Writing reducer.py


In [4]:
!mkdir input

In [5]:
%%writefile input/books_dataset.txt
1,Agree.,Jeffrey Wright,2017,Spain,Spanish,269
2,Camera result.,Richard Chandler,2018,Greece,German,884
3,Opportunity left us.,Nathan Nelson,2021,Senegal,English,189
4,Team I.,Renee Yang,2010,Cameroon,French,513
5,Note represent.,Nathan Joyce,2020,Qatar,French,861
6,Focus suddenly past.,Suzanne Dawson,2016,Malawi,Italian,753
7,Boy bad.,Rachel Mclean,2023,Dominica,Italian,583
8,Reduce bar may resource.,Jennifer Moore,2024,Saint Pierre and Miquelon,German,364
9,On see join.,Denise Hanson,2019,Belarus,Spanish,380
10,Soon front include.,Todd Sanchez,2023,Zimbabwe,Spanish,334
11,Expert us.,Katelyn Alvarado,2024,Yemen,Spanish,121
12,Think specific system.,Jill Soto,2014,Turks and Caicos Islands,Spanish,926
13,Understand Congress.,Hayley Johnson,2019,Cote d'Ivoire,Spanish,899
14,Simply business.,David Estrada,2013,Saint Vincent and the Grenadines,German,313
15,Take baby.,Tara Stokes,2024,Falkland Islands (Malvinas),English,426
16,Good century research.,Maxwell Green,2021,Bulgaria,Spanish,780
17,Our imagine effort.,Jacqueline White PhD,2012,Belgium,Spanish,657
18,Prove describe individual.,Stephen White,2016,United Kingdom,Italian,337
19,High.,Anthony Moore,2010,Saint Helena,Italian,987
20,Toward learn.,Larry Waters,2015,Faroe Islands,English,765
21,Image available can.,Pamela Bond,2019,Guam,Italian,498
22,Camera nor.,Jennifer Wright,2024,Guinea,Italian,695
23,These hospital apply.,Justin Anderson,2010,Saint Helena,French,60
24,Knowledge final.,Rodney Lewis,2021,Norfolk Island,German,181
25,Human ask both.,Stephen Cain,2023,British Virgin Islands,Italian,109
26,Pretty.,David Lynch,2010,French Polynesia,German,119
27,Bag nearly.,Anne Pacheco,2015,Belize,Italian,460
28,Occur knowledge science.,Bethany Doyle,2020,Spain,French,819
29,Issue sell.,Jennifer Woodward,2021,Netherlands,German,423
30,Well small near.,Mrs. Angela Arellano,2013,Samoa,German,377
31,Key can.,Kimberly Brown,2014,Nigeria,Spanish,491
32,Operation three.,Lindsay Hamilton,2021,Bhutan,Spanish,525
33,Begin.,Michael Webb,2015,French Guiana,Spanish,794
34,Area under name.,Laura Smith,2010,Aruba,English,646
35,Style without challenge.,David Jones,2010,Monaco,Italian,186
36,Personal other draw.,Julie Wood,2017,Western Sahara,French,793
37,Easy.,Paul Reynolds,2021,Afghanistan,French,830
38,Soldier fear resource.,James Gould,2017,Greenland,German,98
39,Cut similar.,Megan Torres,2017,El Salvador,French,378
40,Turn five put.,Debbie Gould,2024,Algeria,English,610
41,End former.,Marcus Herring,2021,Norfolk Island,English,853
42,See wide benefit.,Julie Paul,2014,Saudi Arabia,Italian,906
43,Day forget.,Sheila Johnson,2018,Germany,German,672
44,Deep thousand consider.,Bruce Page,2013,Swaziland,French,855
45,Staff idea simple.,Heidi Thomas,2019,Guernsey,German,751
46,Continue no.,Stephanie Franklin,2014,Mauritania,English,63
47,Morning company benefit.,Kathleen Lopez,2012,Switzerland,Spanish,684
48,Member nice.,Richard Wallace DDS,2016,Denmark,French,426
49,Consider early.,Greg Harris,2014,Belarus,German,841
50,Foreign include bag.,Christopher Lara,2024,Bhutan,German,341

Writing input/books_dataset.txt


In [6]:
!hadoop jar /content/hadoop-3.3.6/share/hadoop/tools/lib/hadoop-streaming-3.3.6.jar \
    -files mapper.py,reducer.py \
    -mapper mapper.py \
    -reducer reducer.py \
    -input input \
    -output output

2024-04-14 16:17:27,106 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2024-04-14 16:17:27,196 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2024-04-14 16:17:27,196 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2024-04-14 16:17:27,231 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2024-04-14 16:17:27,770 INFO mapred.FileInputFormat: Total input files to process : 1
2024-04-14 16:17:27,796 INFO mapreduce.JobSubmitter: number of splits:1
2024-04-14 16:17:28,128 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local1830794369_0001
2024-04-14 16:17:28,128 INFO mapreduce.JobSubmitter: Executing with tokens: []
2024-04-14 16:17:28,828 INFO mapred.LocalDistributedCacheManager: Localized file:/content/mapper.py as file:/tmp/hadoop-root/mapred/local/job_local1830794369_0001_f9d03634-a9a7-44ad-a960-ce80ca14b89a/mapper.py
2024-04-14 16:17:28,853 INFO mapred.LocalDistributedCacheMa

In [7]:
!cat output/part-00000

Year	Count
	
2010	6
2012	2
2013	3
2014	5
2015	3
2016	3
2017	4
2018	2
2019	4
2020	2
2021	7
2023	3
2024	6
	
Year with maximum books published: 2021 (no.of.books : 7)	
